In [1]:
#%load_ext autoreload
#%autoreload 1
#%aimport graph_description
import networkx as nx
import pysubgroup as ps
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
from graph_description.datasets import read_attributed_graph
from graph_description.utils import prune_sparse_selectors
from graph_description.networkx_aggregation import SumAggregator, MeanAggregator, apply_aggregator    

In [3]:
from graph_description.datasets import create_random_split

In [4]:
G, df = read_attributed_graph("citeseer", group="planetoid")

In [5]:
train_mask, val_mask, test_mask = create_random_split(df, 100, 500, "rest")

In [6]:
df_labels = df[["labels"]]

In [7]:
df_labels["labels"][~train_mask]=-1

C:\Users\felix\AppData\Local\Temp\ipykernel_19460\1281064040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels["labels"][~train_mask]=-1


In [8]:
df_dummies = pd.get_dummies(df_labels,columns=["labels"])
df_dummies

,labels_-1,labels_0,labels_1,labels_2,labels_3,labels_4,labels_5
0,True,False,False,False,False,False,False
1,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False
3,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...
3322,True,False,False,False,False,False,False
3323,True,False,False,False,False,False,False
3324,True,False,False,False,False,False,False
3325,False,False,True,False,False,False,False


In [9]:
searchspace = ps.create_selectors(df_dummies)
searchspace = [sel for sel in searchspace if "==0" not in str(sel)]

In [42]:
%%time
# do the actual propagation
df1 = apply_aggregator(SumAggregator, df_dummies, G, searchspace)
df2 = apply_aggregator((SumAggregator, MeanAggregator), df1, G)
df3 = apply_aggregator((SumAggregator, MeanAggregator), df2, G)

init
prep done
init
prep done
init
prep done
CPU times: total: 15.6 ms
Wall time: 22.2 ms


In [43]:
#create the final dataframe & searchspace
total_df = pd.concat([df1, df2, df3, df["labels"]], axis=1)

In [44]:
#pd.isna(train_df).any(axis=1).sum()

In [45]:
train_df = total_df
train_df.columns= list(map(str, train_df.columns))

In [46]:
from sklearn.tree import DecisionTreeClassifier

In [47]:
clf = DecisionTreeClassifier()

In [48]:
train_df

,neigh_sum(labels_-1==True),neigh_sum(labels_-1==False),neigh_sum(labels_0==False),neigh_sum(labels_0==True),neigh_sum(labels_1==False),neigh_sum(labels_1==True),neigh_sum(labels_2==False),neigh_sum(labels_2==True),neigh_sum(labels_3==False),neigh_sum(labels_3==True),...,neigh_mean(neigh_mean(neigh_sum(labels_2==False))),neigh_mean(neigh_mean(neigh_sum(labels_2==True))),neigh_mean(neigh_mean(neigh_sum(labels_3==False))),neigh_mean(neigh_mean(neigh_sum(labels_3==True))),neigh_mean(neigh_mean(neigh_sum(labels_4==False))),neigh_mean(neigh_mean(neigh_sum(labels_4==True))),neigh_mean(neigh_mean(neigh_sum(labels_5==False))),neigh_mean(neigh_mean(neigh_sum(labels_5==True))),neigh_mean(neigh_mean(neigh_sum(all_ones))),labels
0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.000000,0.000000,0.000000,1.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
1,4.0,1.0,5.0,0.0,4.0,1.0,5.0,0.0,5.0,0.0,...,8.546667,0.542982,9.089649,0.0,9.079123,0.010526,9.089649,0.000000,9.089649,1
2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,5
3,1.0,1.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,...,2.250000,0.000000,2.250000,0.0,2.250000,0.000000,1.416667,0.833333,2.250000,5
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3322,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
3323,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
3324,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,...,14.055556,1.611111,15.666667,0.0,15.583333,0.083333,15.666667,0.000000,15.666667,3
3325,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,1


In [49]:
clf.fit(train_df[train_mask].drop("labels", axis=1), train_df[train_mask]["labels"])

DecisionTreeClassifier()

In [50]:
from sklearn.metrics import accuracy_score

In [71]:
train_df

,neigh_sum(labels_-1==True),neigh_sum(labels_-1==False),neigh_sum(labels_0==False),neigh_sum(labels_0==True),neigh_sum(labels_1==False),neigh_sum(labels_1==True),neigh_sum(labels_2==False),neigh_sum(labels_2==True),neigh_sum(labels_3==False),neigh_sum(labels_3==True),...,neigh_mean(neigh_mean(neigh_sum(labels_2==False))),neigh_mean(neigh_mean(neigh_sum(labels_2==True))),neigh_mean(neigh_mean(neigh_sum(labels_3==False))),neigh_mean(neigh_mean(neigh_sum(labels_3==True))),neigh_mean(neigh_mean(neigh_sum(labels_4==False))),neigh_mean(neigh_mean(neigh_sum(labels_4==True))),neigh_mean(neigh_mean(neigh_sum(labels_5==False))),neigh_mean(neigh_mean(neigh_sum(labels_5==True))),neigh_mean(neigh_mean(neigh_sum(all_ones))),labels
0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.000000,0.000000,0.000000,1.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
1,4.0,1.0,5.0,0.0,4.0,1.0,5.0,0.0,5.0,0.0,...,8.546667,0.542982,9.089649,0.0,9.079123,0.010526,9.089649,0.000000,9.089649,1
2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,5
3,1.0,1.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,...,2.250000,0.000000,2.250000,0.0,2.250000,0.000000,1.416667,0.833333,2.250000,5
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3322,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
3323,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,3
3324,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,...,14.055556,1.611111,15.666667,0.0,15.583333,0.083333,15.666667,0.000000,15.666667,3
3325,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,1


In [70]:
for i in range(10):
    clf2 = DecisionTreeClassifier()
    clf2.fit(train_df[train_mask].drop("labels", axis=1), train_df[train_mask]["labels"])
    print(accuracy_score(train_df[["labels"]][test_mask], clf2.predict(train_df.drop("labels", axis=1)[test_mask])))
    print(accuracy_score(train_df[["labels"]][train_mask], clf2.predict(train_df.drop("labels", axis=1)[train_mask])))
    print()

0.33363268971710824
0.9866666666666667

0.3349797934440952
0.9866666666666667

0.38302649303996406
0.9866666666666667

0.3349797934440952
0.9866666666666667

0.33632689717108216
0.9866666666666667

0.33408172429277055
0.9866666666666667

0.33542882801975754
0.9866666666666667

0.33587786259541985
0.9866666666666667

0.4005388414907948
0.9866666666666667

0.3349797934440952
0.9866666666666667



In [52]:
from sklearn.linear_model import SGDClassifier

In [69]:
for i in range(10):
    clf2 = SGDClassifier()
    clf2.fit(train_df[train_mask].drop("labels", axis=1), train_df[train_mask]["labels"])
    print(accuracy_score(train_df[["labels"]][test_mask], clf2.predict(train_df.drop("labels", axis=1)[test_mask])))
    print(accuracy_score(train_df[["labels"]][train_mask], clf2.predict(train_df.drop("labels", axis=1)[train_mask])))
    print()

0.41850022451728786
0.6183333333333333

0.5307588684328693
0.7583333333333333

0.39964077233947015
0.6483333333333333

0.28738212842388866
0.515

0.3888639425235743
0.63

0.44499326448136506
0.6416666666666667

0.5177368657386618
0.7833333333333333

0.4557700942972609
0.6966666666666667

0.4939380332285586
0.6983333333333334

0.42568477772788504
0.6366666666666667



In [54]:
#bst = XGBClassifier(n_estimators=10, max_depth=2, learning_rate=1, objective='binary:logistic')
## fit model
#bst.fit(X_train, y_train)

In [55]:
#bst.score(X_test, y_test)

In [22]:
#from sklearn.linear_model import LogisticRegression
#clf_LR = LogisticRegression()
#cross_val_score(clf_LR, train_df, total_df['label'], cv=5)

## For comparison

From Prediction instability paper:
### Citeseer

GAT: Accuracy 69.04+-0.88

GCB: Accuracy 69.11+- 0.61


## xgboost

In [23]:
%%time
clf = DecisionTreeClassifier
cross_val_score(clf, train_df, total_df['label'], cv=5)

NameError: name 'cross_val_score' is not defined

In [24]:
%%time
clf = XGBClassifier(n_estimators=15, max_depth=3, learning_rate=1, objective='binary:logistic')
cross_val_score(clf, train_df, total_df['label'], cv=5)

NameError: name 'XGBClassifier' is not defined

In [25]:
%%time
clf = XGBClassifier(n_estimators=20, max_depth=3, learning_rate=1, objective='binary:logistic')
cross_val_score(clf, train_df, total_df['label'], cv=5)

NameError: name 'XGBClassifier' is not defined

In [26]:
%%time
clf_RF = XGBRFClassifier(n_estimators=20, max_depth=3, learning_rate=1, objective='binary:logistic')
cross_val_score(clf_RF, train_df, total_df['label'], cv=5)

NameError: name 'XGBRFClassifier' is not defined

## sklearn

In [27]:
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
import warnings

In [28]:
%%time
dummy_clf = DummyClassifier()
cross_val_score(dummy_clf, train_df, total_df['label'], cv=5)

NameError: name 'cross_val_score' is not defined

In [29]:
%%time
clf_NB = GaussianNB()
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.")
    cross_val_score(clf_NB, train_df, total_df['label'], cv=5)

NameError: name 'cross_val_score' is not defined

In [30]:
pd.isna(train_df).any(axis=1)

0       False
1       False
2       False
3       False
4       False
        ...  
3322    False
3323    False
3324    False
3325    False
3326    False
Length: 3327, dtype: bool